# 基于 GCN 的有监督学习

图神经网络（GNN）结合了图结构和机器学习的优势. GraphScope提供了处理学习任务的功能。本次教程，我们将会展示GraphScope如何使用GCN算法训练一个模型。

本次教程的学习任务是在文献引用网络上的点分类任务。在点分类任务中，算法会确定[Cora](https://linqs.soe.ucsc.edu/data)数据集上每个顶点的标签。在```Cora```数据集中，由学术出版物作为顶点，出版物之间的引用作为边，如果出版物A引用了出版物B，则图中会存在一条从A到B的边。Cora数据集中的节点被分为了七个主题类，我们的模型将会训练来预测出版物顶点的主题。

在这一任务中，我们使用图聚合网络(GCN)算法来训练模型。有关这一算法的更多信息可以参考["Knowing Your Neighbours: Machine Learning on Graphs"](https://medium.com/stellargraph/knowing-your-neighbours-machine-learning-on-graphs-9b7c3d0d5896)

这一教程将会分为以下几个步骤：

- 启动GraphScope的学习引擎，并将图关联到引擎上
- 使用内置的GCN模型定义训练过程，并定义相关的超参
- 开始训练


In [ ]:
# Install graphscope package if you are NOT in the Playground

!pip3 install graphscope

In [ ]:
# Import the graphscope module.

import graphscope

graphscope.set_option(show_log=False)  # enable logging

In [ ]:
# Load cora dataset

from graphscope.dataset import load_cora_graph

graph = load_cora_graph()

然后，我们需要定义一个特征列表用于图的训练。训练特征集合必须从点的属性集合中选取。在这个例子中，我们选择了属性集合中所有以"feat_"为前缀的属性作为训练特征集，这一特征集也是Cora数据中点的特征集。

借助定义的特征列表，接下来，我们使用 [graphlearn](https://graphscope.io/docs/reference/session.html#graphscope.Session.graphlearn) 方法来开启一个学习引擎。

在这个例子中，我们在 "graphlearn" 方法中，指定在数据中 "paper" 类型的顶点和 "cites" 类型边上进行模型训练。

利用 "gen_labels" 参数，我们将 "paper" 点数据集进行划分，其中75%作为训练集，10%作为验证集，15%作为测试集。

In [ ]:
# define the features for learning
paper_features = []
for i in range(1433):
    paper_features.append("feat_" + str(i))

# launch a learning engine.
lg = graphscope.graphlearn(graph,
    nodes=[("paper", paper_features)],
    edges=[("paper", "cites", "paper")],
    gen_labels=[
        ("train", "paper", 100, (0, 75)),
        ("val", "paper", 100, (75, 85)),
        ("test", "paper", 100, (85, 100))
    ]
)


这里我们使用内置的GCN模型定义训练过程。你可以在[Graph Learning Model](https://graphscope.io/docs/learning_engine.html#data-model)获取更多内置学习模型的信息。

在本次示例中，我们使用tensorflow作为NN后端训练器。

In [ ]:
from graphscope.learning.examples import GCN
from graphscope.learning.graphlearn.python.model.tf.trainer import LocalTFTrainer
from graphscope.learning.graphlearn.python.model.tf.optimizer import get_tf_optimizer

# supervised GCN.

def train(config, graph):
    def model_fn():
        return GCN(
            graph,
            config["class_num"],
            config["features_num"],
            config["batch_size"],
            val_batch_size=config["val_batch_size"],
            test_batch_size=config["test_batch_size"],
            categorical_attrs_desc=config["categorical_attrs_desc"],
            hidden_dim=config["hidden_dim"],
            in_drop_rate=config["in_drop_rate"],
            neighs_num=config["neighs_num"],
            hops_num=config["hops_num"],
            node_type=config["node_type"],
            edge_type=config["edge_type"],
            full_graph_mode=config["full_graph_mode"],
        )
    trainer = LocalTFTrainer(
        model_fn,
        epoch=config["epoch"],
        optimizer=get_tf_optimizer(
            config["learning_algo"], config["learning_rate"], config["weight_decay"]
        ),
    )
    trainer.train_and_evaluate()

# define hyperparameters
config = {
    "class_num": 7,  # output dimension
    "features_num": 1433,
    "batch_size": 140,
    "val_batch_size": 300,
    "test_batch_size": 1000,
    "categorical_attrs_desc": "",
    "hidden_dim": 128,
    "in_drop_rate": 0.5,
    "hops_num": 2,
    "neighs_num": [5, 5],
    "full_graph_mode": False,
    "agg_type": "gcn",  # mean, sum
    "learning_algo": "adam",
    "learning_rate": 0.01,
    "weight_decay": 0.0005,
    "epoch": 5,
    "node_type": "paper",
    "edge_type": "cites",
}


在定义完训练过程和超参后，现在我们可以使用学习引擎和定义的超参开始训练过程。


In [ ]:
train(config, lg)